In [ ]:
# Session identifier, do not remove or modify.
from yields.session import Session
session = Session("b87848c9-a30f-4938-9de8-293be09d36bb")

In [ ]:
from yields.providers.filesystem import ProviderFactoryFilesystem
from yields.providers.hdfs import ProviderFactoryHdfs

In this notebook, we train a simple benchmark and save it inside the platform. The main reason to do so, is that this benchmark model can be retrieved during the model validation of the model that is handed over to the model assesor. One of the most independent manners to challenge a given model, is to build your own challenger model and apply it to the same application data. <br>
Any discrepancy between the two might indicate possible shortcomings/weaknesses.

It is considered best practice to save the trained model (as well as some preprocessing steps), in particular when using large datasets. Reasons for this are:
* the dataset themselves should not be shared
* the preprocessing should not be recalculated on a equally large dataset, thus saving computational resources. This saves time in case any other party would start training a different model, but using your preprocessed data.
* the saved benchmark model is well-defined in terms of auditability/reproducibility and can be immediately applied onto the application dataset.


<a id="toc"></a>
## Table of Contents
1. [Setup](#set)     

    1.1 [Import Libraries](#imp_lib)  
    
    1.2 [Load input](#load_input)   
      
    
2. [Train/test split by stratified sampling](#data_split)

3. [Training a logistic regression model using scikitlearn](#model_train)       
    
4. [Generalization error of the trained model](#gen_error) 
    
5. [Saving the benchmark model](#model_save)  

-------------------------------------------------------------------------------------

<a id="set"></a>
# 1. [Setup](#toc)
---
<a id="imp_lib"></a>
## 1.1. [Import Libraries](#toc)

In [ ]:
# To read and write to and from platform + analytics support - see API documentation
from yields.providers.filesystem import ProviderFactoryFilesystem

# standard python libraries
import json, os , sklearn
import numpy as np
import pandas as pd
import ipywidgets as widgets
from IPython.core.display import HTML
import sklearn.metrics as sk_metrics
import scipy.stats as stats
import sklearn
import pickle

# plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

#misc
import datetime

# spark = io.spark
# spark.sparkContext.setLogLevel('ERROR')


<a id="load_input"></a>
## 1.2. [Load dataset](#toc)

In [ ]:
#Retrieve input, output, parameter aliases
session.io.inputs, session.io.outputs, session.io.parameters

In [ ]:
#Dataset
artifact = session.io.input('ds')
dataset_file_record = artifact.file(artifact.files[0]).open()  #Get first file in the given artifact version - User could also pass name of file directly artifact.file('ppnr.csv').open()
#Read the inputs using existing open source methods
raw_data = pd.read_csv(dataset_file_record)

#Data model mapping
artifact_dmm = session.io.input('dmm_ds')
dmm_file_record = artifact_dmm.file(artifact_dmm.files[0]).open()  #Get first file in the given artifact version - User could also pass name of file directly artifact.file('ppnr.csv').open()
schema = json.load(dmm_file_record)

<a id="data_split"></a>
# 2. [Train/test split by stratified sampling](#toc)

Define benchmark model inputs. For sake of illustration we take the same inputs as for the incumbent models. This means we for example do not do any stepwise regression here or look for better model variables. <br>
Off course, it's up the user how far he/she wants to go here, but in the end the result will be a (scikit learn) trained model which we store inside a freeform artifact. <br>
This model in turn will then be used during model validation where its performance is assessed against the incumbent model.

In [ ]:
categorical_model_input = schema["categorical_inputs"]
continuous_model_input = schema["continuous_inputs"]
target = schema["target"]
model_input = categorical_model_input + continuous_model_input

In [ ]:
#only select model inputs + target: note that we do not consider possible other risk drivers - as this script is solely for illustration purposes to create a simple benchmark model and save it.
model_data = raw_data[model_input + [target]]

In [ ]:
#Assuming we have a strongly imbalanced target (i.e. say only 1% of the fulll dataset has target =1), we split our full dataset into two disjunct datasets (train and test), but that bot contain a sufficient number of target =1.
#If we were to sample uniformly, chances would be high that we only have target = 0 in our dataset and thus it is impossible to create a model for this.
#By performing stratified sampling, with the target as stratum, we ensure that the proportion of target in both train and test set are of the same order and we thus ensure sufficient target = 1 to train the model.
train_data = model_data.groupby(target, group_keys=False).apply(lambda x: x.sample(frac=0.8 , random_state = 1))
test_data = model_data[~model_data.index.isin(train_data.index)] #This test data is only used inside this scripts and is needed to assess the benchmark model performance internally, i.e. are we not overfitting on the train data.


In [ ]:
#Check the average target rate in both train and test set
df_ratios = pd.DataFrame(index = ['Target rate'] ,columns = ['Total' , 'Train' , 'Test'])
df_ratios.loc['Target rate']['Train'] = train_data[target].astype('float').mean()
df_ratios.loc['Target rate']['Test'] = test_data[target].astype('float').mean()
df_ratios.loc['Target rate']['Total'] = model_data[target].astype('float').mean()
display(df_ratios)

<a id="model_train"></a>
# 3. [Training a logistic regression model using scikitlearn](#toc)   

## SCALING THE CONTINUOUS MODEL INPUTS TO THE RANGE [-1, 1]

In [ ]:
import sklearn.preprocessing as sk_pp

In [ ]:
#Scaling continuous inputs to 'order of magnitude 1': this enables better and faster learning. In particular relevant when working with neural networks.
scaler = sk_pp.StandardScaler().fit(train_data[continuous_model_input])

In [ ]:
#Check whether the transformation indeed works fine
train_data_scaled = scaler.transform(train_data[continuous_model_input])
df_ = pd.DataFrame(data = train_data_scaled, index = train_data.index, columns = continuous_model_input)
#print mean and stdev of transformed variables (should be all 0 and all 1)
display(df_.describe().loc[['mean','std'],:])

In [ ]:
#Let's also apply the fitted preprocessor to our out-of-sample test set.
df_test_scaled_continous = pd.DataFrame(data = scaler.transform(test_data[continuous_model_input]) , index = test_data.index,  columns = continuous_model_input)
df_test_scaled_continous.head()

## ONE-HOT ENCODING OF THE CATEGORICAL MODEL VARIABLES

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
#Fit a one-hot-encoder to the training data. This Object will be saved for further downstream analysis or in case the user wants to share it with others (particulary useful if one uses larger datasets. In this way, the dataset itself should not be shared).
enc = OneHotEncoder(handle_unknown='ignore' , sparse=False)
OHE = enc.fit(train_data[categorical_model_input])

In [ ]:
#Check whether the transformation indeed works fine
df_OHE_Transformed = OHE.transform(train_data[categorical_model_input])

df_cat = pd.DataFrame(data = df_OHE_Transformed , index = train_data.index , columns = OHE.get_feature_names_out())

#Also apply the fitted one-hot encoder to the test data.
df_test_cat = pd.DataFrame(data = OHE.transform(test_data[categorical_model_input]) , index = test_data.index , columns = OHE.get_feature_names_out())
df_cat.head()

In [ ]:
#Lets' show an example of preprocessed train data
pd.set_option('display.max_columns', None)

In [ ]:
df_train_preprocessed = df_.join(df_cat)
df_train_preprocessed = df_train_preprocessed.join(model_data[target])
df_test_preprocessed = df_test_scaled_continous.join(df_test_cat)
df_test_preprocessed = df_test_preprocessed.join(model_data[target])

In [ ]:
df_train_preprocessed.head(5)

In [ ]:
#Show the original data
train_data.head(5)

In [ ]:
print("Shape of training data BEFORE preprocessing: " + str(train_data.shape) )
print("Shape of training data AFTER preprocessing: " + str(df_train_preprocessed.shape) )
print("Shape of test data BEFORE preprocessing: " + str(test_data.shape) )
print("Shape of test data AFTER preprocessing: " + str(df_test_preprocessed.shape) )
print(df_test_preprocessed.columns)

<a id="gen_error"></a>
# 4. [Training and assessing the benchmark model](#toc) 

## FIT A LOGISTIC REGRESSION

In [ ]:
from sklearn.linear_model import LogisticRegression

X, y  = df_train_preprocessed.drop(target, axis = 1) , df_train_preprocessed[target] 
benchmark_model = LogisticRegression(random_state=0).fit(X,y)


## USE THE TRAINED LOGISTIC REGRESSION TO PREDICT PROBABILITY OUTCOMES ON TRAIN AND TEST SET

In [ ]:
#Predict probabilities on the training set and valiation (test set). Two columns are returned, one the complement of the other. TO see which column indicates the probability for what target, use clf.classes_

prob_train = benchmark_model.predict_proba(X)[:,1]

X_test = df_test_preprocessed.drop(target, axis = 1) 
prob_test = benchmark_model.predict_proba(X_test)[:,1]
y_test =  df_test_preprocessed[target]

## ASSESS THE BENCHMARK MODEL PERFORMANCE ON TRAIN AND TEST SET (e.g. DISCRIMINATORY POWER)

In [ ]:
""" ROC """
from plotly.subplots import make_subplots

fpr, tpr, _ = sklearn.metrics.roc_curve(y,  prob_train)
fpr_test, tpr_test, _ = sklearn.metrics.roc_curve(y_test,  prob_test)

fig = make_subplots()
fig.update_layout(title_text = 'ROC-curve')
fig.update_xaxes(title_text = 'TPR')
fig.update_yaxes(title_text = 'FPR')
fig.add_trace(go.Scatter(x = fpr , y = tpr  , name = 'train'))
fig.add_trace(go.Scatter(x = fpr_test , y = tpr_test  , name = 'test'))
fig.add_trace(go.Scatter(x = [0,1] , y = [0,1] , name = 'random' ))

fig.show()

auc_score = sk_metrics.roc_auc_score(y, prob_train)
auc_score_test = sk_metrics.roc_auc_score(y_test, prob_test)
print('Area-under-the-ROC-curve on TRAIN SET: ' + str(auc_score))
print('Area-under-the-ROC-curve on TEST SET: ' + str(auc_score_test))

<a id="model_save"></a>
# 5. [Saving the benchmark model](#toc)  

## Let's save the standardscaler, the one-hot-encoder and the trained benchmark classifier into three different artifacts.

Off course, scikit learn allows for 'pipeline objects' (that contain both the preprocessor and the trained classifier) which can also be saved. This notebooks therefore also shows how the user can save as many artifacts he wants. In practice, a stand-alone preprocessor is less useful.

In [ ]:
session.io.outputs #To remember the notebook aliases to be used.

In [ ]:
# Call the output artifact in which user wants to register a version
artifact_1 = session.io.output('benchmark_model')
file_name = 'Benchmark_classifier.pickle'
output1_test = artifact_1.file(file_name, ProviderFactoryFilesystem()) # StringIO buffer
pickle.dump(benchmark_model, output1_test)
artifact_1.save()

In [ ]:
artifact_1.name

In [ ]:
# Call the output artifact in which user wants to register a version
artifact_1 = session.io.output('Continuous_scaler')
file_name = 'continuous_scaler.pickle'
output1_test = artifact_1.file(file_name, ProviderFactoryFilesystem()) # StringIO buffer
pickle.dump(scaler, output1_test)
artifact_1.save()

In [ ]:
artifact_1.name

In [ ]:
# Call the output artifact in which user wants to register a version
artifact_1 = session.io.output('Categorical_encoder')
file_name = 'Categorical_encoder.pickle'
output1_test = artifact_1.file(file_name, ProviderFactoryFilesystem()) # StringIO buffer
pickle.dump(OHE, output1_test)
artifact_1.save()

In [ ]:
artifact_1.name